In [181]:
import pandas as pd
import numpy as np
import os

try:
    

    df = pd.read_excel('/Users/rajeshpillai/Downloads/Online_Retail.xlsx',skiprows=0,header=0)

    df = df.dropna()

except:
    print('File Not Found')
    
# Add Total by multiplying Quantity and Unit Price

df['Total'] = df.Quantity*df.UnitPrice

# Change date format to YYYYMM

df['DateYMD'] = df.InvoiceDate.dt.strftime('%Y%m%d')

df_customer = df.copy(deep=True)

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total,DateYMD
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,20101201
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,20101201
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201


### Customer Lifetime Value Model

  #### Calculate average purchase value: 
  
        Calculate this number by dividing your company's total revenue in a time period (usually one year)
        by the number of purchases over the course of that same time period.
  #### Calculate average purchase frequency rate: 
  
      Calculate this number by dividing the number of purchases by the number of unique customers who made
      purchases during that time period.
      
   #### Calculate customer value: 
   
       Calculate this number by multiplying the average purchase value by the average purchase frequency rate.
       
   #### Calculate average customer lifespan: 
   
       Calculate this number by averaging the number of years a customer continues purchasing from your company.
       
   #### Calculate CLTV: 
   
       multiply customer value by the average customer lifespan. This will give you the revenue you can
       reasonably expect an average customer to generate for your company over the course of their relationship
       with you.


In [182]:
# Select Country, Year

df = df[df.InvoiceDate.dt.strftime('%y')=='10']

df = df[df.Country=='United Kingdom']

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total,DateYMD
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,20101201
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,20101201
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20101201


In [183]:
# Average purchase value

total_revenue_year = df.Total.sum()

no_of_purchase_year = len(df.InvoiceNo.unique())

print(no_of_purchase_year)

# avg_purchase_value
avg_purchase_value = (total_revenue_year/no_of_purchase_year).round(2)

print(avg_purchase_value)

# Average purchase frequency rate

no_of_customers = len(df.CustomerID.unique())

print(no_of_customers)

#avg_purchase_freq
avg_purchase_freq = (no_of_purchase_year/no_of_customers)

print(avg_purchase_freq)

# Calculate Customer Value

avg_customer_value = (avg_purchase_value * avg_purchase_freq).round(2)

print(avg_customer_value)




1572
307.76
871
1.8048220436280138
555.45


In [184]:
# Customer Life Span

df_cust_life_span = df_customer[['CustomerID','InvoiceDate']]

df_cust_life_span = df_cust_life_span.drop_duplicates(keep='first')

df_cust_life_span.CustomerID = df_cust_life_span.CustomerID.astype('int')

df_cust_life_span.count()

df_cust_life_span = df_cust_life_span.groupby('CustomerID').agg(
                                            {
                                            'InvoiceDate' :['first','last']
                                            }
                                            )

df_cust_life_span = df_cust_life_span.reset_index(level=0, col_level=1)\
                                     .rename(columns={'first':'start_dt','last':'end_dt'})\
                                     .droplevel(level=0,axis=1)

df_cust_life_span.head()

# Avg customer life span

df_cust_life_span['avg'] = ((df_cust_life_span.end_dt - df_cust_life_span.start_dt)/2).astype('timedelta64[D]')

#df_cust_life_span.head()

avg_customer_lifespan = round(df_cust_life_span.avg.mean()/365,2)

avg_customer_lifespan

# Customer Life Time Value

CLTV = round(avg_customer_lifespan * avg_customer_value,2)

CLTV

99.98

#### End of Code